In [6]:
import shutil
import os
from random import sample
import random
import json
import re
import pandas as pd


path_train = "D:/code/train wav"
path_meta = "D:/code/metadata"

train = os.listdir(path_train)
meta = os.listdir(path_meta)
labels_dest = "D:/code/kospeech-latest/dataset/kspon"

def rule(x):
    # 괄호
    a = re.compile(r'\([^)]*\)')
    # 문장 부호
    b = re.compile('[^가-힣0-9 ]')
    x = re.sub(pattern=a, repl='', string= x)
    x = re.sub(pattern=b, repl='', string= x)
    return x

def load_label(filepath):
    char2id = dict()
    id2char = dict()

    ch_labels = pd.read_csv(filepath, encoding="utf-8")

    id_list = ch_labels["id"]
    char_list = ch_labels["char"]
    freq_list = ch_labels["freq"]

    for (id_, char, freq) in zip(id_list, char_list, freq_list):
        char2id[char] = id_
        id2char[id_] = char
    return char2id, id2char

char2id, id2char = load_label(os.path.join(labels_dest, "cssiri_character_vocabs.csv"))

def sentence_to_target(sentence, char2id):
    target = str()

    for ch in sentence:
        try:
            target += (str(char2id[ch]) + ' ')
        # 사전에 없는 경우 넘어가라 -> 그냥 묵음처리나 마찬가지.
        except KeyError:
            continue

    return target[:-1]

seeds = [777, 55, 555, 5555, 7]
for n in range(5):
    random.seed(seeds[n])
    randints = sorted(sample(range(0,300000),10000))
    dst = f'D:/code/sample{n+1}'
    if not os.path.isdir(dst):
        os.mkdir(dst)
    elif os.path.isdir(dst):
        shutil.rmtree(dst)
        os.mkdir(dst)
    if not os.path.isdir(os.path.join(dst, "audio")):
        os.mkdir(os.path.join(dst, "audio"))
    for num in randints:
        shutil.copy(os.path.join(path_train, train[num]), os.path.join(dst, "audio"))
        with open(path_meta + "/" + meta[num], 'r', encoding='UTF-8') as f:

            json_data = json.load(f)
            reading = json.dumps(json_data["transcription"]["ReadingLabelText"], ensure_ascii = False)
            reading = rule(reading)
            char_id_transcript = sentence_to_target(reading, char2id)
        with open(os.path.join(dst,"transcripts.txt"), "a") as f:
            f.write(f'{train[num]}\t{reading}\t{char_id_transcript}\n')

In [3]:
bool('a')

True